In [ ]:
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")

# connect notebook to db
conn = sqlite3.connect('hp-dataset-with-relations.sqlite')
c = conn.cursor()

# define function to get sentiment of a table
def calculate_sentiment(text):
    analysed = [nlp(line) for line in text]
    polarities = [l._.polarity for l in analysed]
    assessments = [l._.assessments for l in analysed]
    #return assessments
    return polarities
    

# get namelist of all available views in database
views = c.execute("SELECT tbl_name FROM sqlite_master where type='view'").fetchall()
views_names = [table[0] for table in views] 
views_names.remove('all_parts')

# test single table
sql = c.execute("SELECT dialog FROM harry_voldemort").fetchall()  
dialog = [i[0] for i in sql]
res = calculate_sentiment(dialog)
print(res)

#print(sql)

# call function with every view
for table in views_names:
    sql = c.execute("SELECT dialog FROM " + table).fetchall()  
    dialog = [i[0] for i in sql]
    result = calculate_sentiment(dialog)
    narr = np.array(result)
    mean = np.mean(narr)
    #print(table + ': || ' + str(mean) + ' ||' + str(result))
    #plt.clf()
    
    plt.title(table + ' ' + str(mean))
    x_axis = (-1, 1)
    if table.startswith('harry'):
        plt.hist(narr, range = x_axis, color='g')
    else:
        plt.hist(narr, range = x_axis, color='r')
        
    plt.savefig(table + '.jpg')

[0.0, 0.0, -0.25625, 0.0, -0.20833333333333334, 0.15, -0.05, 0.0, 0.0, -0.3125, 0.0, 0.0, 0.21428571428571427, 0.0, 0.0, 0.25, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5, -0.2, 0.05, 0.0, -0.3, 0.1375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.07083333333333335, 0.43333333333333335, 0.0, 0.0, -0.07999999999999999, -0.2666666666666666, 0.0, -0.2, 0.0]
